In [ ]:
simdir = "/mnt/scratch/srichers/NCSU_mergercompare/NSM2.5_matchevan_long_diagonalpert"
emudir = "/mnt/scratch/srichers/software/Emu"
plotfile = "plt90000"

import os
os.environ['HDF5_USE_FILE_LOCKING'] = 'FALSE'
import numpy as np
import matplotlib.pyplot as plt
import glob
import h5py
import matplotlib as mpl
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,AutoMinorLocator,LogLocator)
import sys
sys.path.append(emudir+"/Scripts/data_reduction")
import reduce_data as rd
import amrex_plot_tools as amrex
import emu_yt_module as emu

import importlib
importlib.reload(rd)

################
# plot options #
################
mpl.rcParams['font.size'] = 22
mpl.rcParams['font.family'] = 'serif'
#mpl.rc('text', usetex=True)
mpl.rcParams['xtick.major.size'] = 7
mpl.rcParams['xtick.major.width'] = 2
mpl.rcParams['xtick.major.pad'] = 8
mpl.rcParams['xtick.minor.size'] = 4
mpl.rcParams['xtick.minor.width'] = 2
mpl.rcParams['ytick.major.size'] = 7
mpl.rcParams['ytick.major.width'] = 2
mpl.rcParams['ytick.minor.size'] = 4
mpl.rcParams['ytick.minor.width'] = 2
mpl.rcParams['axes.linewidth'] = 2

Plot the average numbers of each flavor of neutrino and the amplitude of the off-diagonals. Requires the file reduced0D.dat

In [ ]:
xlabel = "step" #"time(s)" # "step"
ylabel = "N_offdiag_mag(1|ccm)" # "N00(1|ccm)" #

######################
# read averaged data #
######################
# get the column labels
filename = simdir+"/reduced0D.dat"
with open(filename,"r") as f:
    headers = f.readline().strip().split()
print(headers)

# read the data and fill a dictionary
data = np.genfromtxt(filename, skip_header=1)
datadict = {}
for i,h in enumerate(headers):
    headername = h.split(":")[1]
    datadict[headername] = data[:,i]

##############
# formatting #
##############
fig, ax = plt.subplots(1,1, figsize=(6,5))
#ax.axhline(1./3., color="green")
ax.set_ylabel(ylabel)
ax.set_xlabel(xlabel)
ax.tick_params(axis='both', which='both', direction='in', right=True,top=True)
ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.yaxis.set_minor_locator(AutoMinorLocator())
ax.minorticks_on()
ax.grid(which='both')
ax.semilogy(datadict[xlabel], datadict[ylabel])
plt.show()

Plot data contained in plot files

In [ ]:
dirname = simdir+"/"+plotfile

directories_badformat = sorted(glob.glob(simdir+"/plt*"))
directories = []
steps = []
for d in directories_badformat:
    if os.path.isdir(d):
        directories.append(d)
        steps.append(int(d.split("plt")[-1]))
directories = [x for _, x in sorted(zip(steps, directories))]

for dirname in directories:
    print(dirname)
    eds = emu.EmuDataset(dirname)
    t = eds.ds.current_time
    ad = eds.ds.all_data()
    rd.make_format_dict("Emu", dirname) # defines a global called format_dict

    dname = rd.dataset_name("N", "", 0, 1, "Re")
    print("Plotting dataset",dname)

    # get the full fft
    # note that k does NOT include the factor of 2 pi
    fft = eds.fourier(dname,nproc=1)
    print()
    print("FFT magnitude shape:", fft.magnitude.shape)
    print("FFT phase shape:", fft.phase.shape)
    print("FFT kz shape:", fft.kz.shape)

    # get the power spectrum
    cleft, cright, ileft, iright, kmid = rd.fft_coefficients(fft)
    power = rd.fft_power(fft, cleft, cright, ileft, iright, kmid)
    print()
    print("kmid shape:", kmid.shape)
    print("FFT power shape:", power.shape)

    fig, ax = plt.subplots(1,1, figsize=(8,6))
    ax.semilogy(kmid, power)
    
    # axis labels
    ax.set_xlabel(r"$k\,(\mathrm{cm}^{-1})$")
    ax.set_ylabel(r"$|fft("+dname+")|^2\,(\mathrm{cm}^{-2})$")
    ax.minorticks_on()
    ax.grid(which='both')
    plt.show()

Do angular analysis

In [ ]:
dirname = simdir+"/"+plotfile

directories_badformat = sorted(glob.glob(simdir+"/plt*"))
directories = []
steps = []
for d in directories_badformat:
    if os.path.isdir(d):
        directories.append(d)
        steps.append(int(d.split("plt")[-1]))
directories = [x for _, x in sorted(zip(steps, directories))]

for dirname in directories:
    print(dirname)
    eds = emu.EmuDataset(dirname)
    t = eds.ds.current_time
    ad = eds.ds.all_data()
    rd.make_format_dict("Emu", dirname) # defines a global called format_dict

    dname = rd.dataset_name("N", "", 0, 0, "Re")
    print("Plotting dataset",dname)

    # get the full fft
    # note that k does NOT include the factor of 2 pi
    fft = eds.fourier(dname,nproc=1)
    print()
    print("FFT magnitude shape:", fft.magnitude.shape)
    print("FFT phase shape:", fft.phase.shape)
    print("FFT kz shape:", fft.kz.shape)

    fig, ax = plt.subplots(1,1, figsize=(8,6))
    ax.semilogy(fft.kz, fft.magnitude)
    
    # axis labels
    ax.set_xlabel(r"$k\,(\mathrm{cm}^{-1})$")
    ax.set_ylabel(r"$|fft("+dname+")|^2\,(\mathrm{cm}^{-2})$")
    ax.minorticks_on()
    ax.grid(which='both')
    plt.show()